In [1]:
import requests
import math
import time
import json

from tqdm import tqdm

In [29]:
## This omits players for some reason!!
# players = []


# pageSize = 100

# headers = {
#         "origin": "https://www.premierleague.com"
# }


# page = 0
# while(True):
#     url = 'https://footballapi.pulselive.com/football/players?pageSize=50&page=%s&compSeasons=418&altIds=true&page=0&type=player&id=-1&compSeasonId=418' % page
#     payload = requests.get(url, headers = { "origin": "https://www.premierleague.com"}).json()
#     pageInfo = payload["pageInfo"]
#     content = payload["content"]
#     players.extend(content)
    
#     if pageInfo["page"] == pageInfo["numPages"]:
#         break
    
#     page += 1
#     time.sleep(0.3)
    
    
    
    
    
# print("Collected %s Player's Information!" % len(players))
    
    

Collected 956 Player's Information!


In [ ]:
# with open("raw_player_data.json", "w") as f:
#     f.write(json.dumps(players))

In [10]:
# with open("raw_player_data.json", "r") as f:
#     players = json.loads(f.read())

In [17]:
def get_epl_table():
    url = 'https://footballapi.pulselive.com/football/standings?compSeasons=418&altIds=true&detail=2&FOOTBALL_COMPETITION=1'
    payload = requests.get(url, headers = { "origin": "https://www.premierleague.com"}).json()
    table = payload["tables"][0]
    return table["entries"]
    


def get_team(teamId: int):
    url = 'https://footballapi.pulselive.com/football/teams/%s/compseasons/418/staff?pageSize=50&compSeasons=418&altIds=true&page=0&type=player' % str(teamId)
    return requests.get(url, headers = { "origin": "https://www.premierleague.com"}).json()

In [24]:
with open("epl_table.json", "w") as f:
    table = get_epl_table()
    f.write(json.dumps(table))

In [25]:
all_players = [get_team(t["team"]["id"]) for t in table]
with open("raw_team_data.json", "w") as f:
    f.write(json.dumps(all_players))

In [6]:
def is_invalid(raw_player):
    return "date" not in raw_player["birth"]


def clean_player(team, raw_player):
    playerId = int(raw_player["id"])
    position = raw_player["info"]["position"]
    nation = raw_player["nationalTeam"]
    dob = raw_player["birth"]["date"]["millis"]
    name = raw_player["name"]
    loan = raw_player["info"].get("loan", False)
    number = raw_player["info"]["shirtNum"]
    currentTeam = team["team"]["name"]

    return {
        "id": playerId,
        "position": position,
        "number": number,
        "nation": nation,
        "dob": dob,
        "name": name,
        "team": currentTeam
    }
    
    
def get_player_stats(cleansed_player):
    url = 'https://footballapi.pulselive.com/football/stats/player/%s?comps=1&compSeasons=418' % str(int(cleansed_player["id"]))
    payload = requests.get(url, headers = { "origin": "https://www.premierleague.com"}).json()

    stats = payload["stats"]
    relevant_stats = {}
    
    stat_name_mapping = {
        "goals": "goals",
        "goal_assist": "assists",
        "appearances": "appearances",
        "mins_played": "mins_played"
    }
    
    for s in stats:
        if s["name"] in stat_name_mapping.keys():
            relevant_stats[stat_name_mapping[s["name"]]] = int(s["value"])
        
    for k in stat_name_mapping.values():
        if k not in relevant_stats:
            relevant_stats[k] = 0
            
    return relevant_stats

In [21]:
augmented_players = []
invalid = []
exceptions = []

for team in tqdm(all_players):
    for player in team["players"]:
    
        try:
            if is_invalid(player):
                invalid.append(player)

            else:
                filtered = clean_player(team, player)
                stats = get_player_stats(filtered)
                stats.update(filtered)
                augmented_players.append(stats)
                time.sleep(0.05)
        except:
            exceptions.append(player)
        
with open("augmented_players.json", "w") as f:
    f.write(json.dumps(augmented_players))

100%|##########| 20/20 [02:14<00:00,  6.74s/it]


In [23]:
print(len(augmented_players))
print(len(invalid))
print(len(exceptions))

642
2
1


In [15]:
[p for p in augmented_players if p["name"]["first"] == "Kyle"]

[{'appearances': 18,
  'assists': 2,
  'mins_played': 1575,
  'goals': 0,
  'id': 3955,
  'position': 'D',
  'number': 2,
  'nation': {'isoCode': 'GB-ENG', 'country': 'England', 'demonym': 'English'},
  'dob': 643852800000,
  'name': {'display': 'Kyle Walker',
   'first': 'Kyle',
   'middle': 'Andrew',
   'last': 'Walker'},
  'team': 'Manchester City'},
 {'appearances': 23,
  'goals': 1,
  'assists': 2,
  'mins_played': 1829,
  'id': 13815,
  'position': 'D',
  'number': 2,
  'nation': {'isoCode': 'GB-ENG', 'country': 'England', 'demonym': 'English'},
  'dob': 860889600000,
  'name': {'display': 'Kyle Walker-Peters',
   'first': 'Kyle',
   'last': 'Walker-Peters'},
  'team': 'Southampton'}]

In [25]:
[p for p in augmented_players if p["name"]["display"] == "Phil Foden"]

[{'appearances': 21,
  'goals': 7,
  'assists': 3,
  'mins_played': 1611,
  'id': 14805,
  'position': 'M',
  'number': 47,
  'nation': {'isoCode': 'GB-ENG', 'country': 'England', 'demonym': 'English'},
  'dob': 959472000000.0,
  'name': {'display': 'Phil Foden', 'first': 'Phil', 'last': 'Foden'},
  'team': 'Manchester City'}]

In [22]:
print([p["name"]["display"] for p in invalid])

['James Storer', 'Shaqai Forde']
